In [1]:
#Importamos las librerias y clases necesarias. Empezando por las de implementación de Hanoi disponibles en repo. 
from aima_libs.hanoi_states import StatesHanoi, ProblemHanoi
from aima_libs.hanoi_states import ActionHanoi
from aima_libs.tree_hanoi import NodeHanoi


In [2]:

def hanoi_busqueda_anchura_no_optimizada(disks=5):
    #Parametrizamos para ir aumentando discos 
    discos=disks

    #Seteamos estado inicial 
    varilla_1 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    varilla_2 = [] 
    varilla_3 = [] 
    estado_inicial = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Seteamos estado final 
    varilla_1 = []
    varilla_2 = [] 
    varilla_3 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    estado_objetivo = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Inicializamos la clase problema de hanoi con estado inicial y objetivo 
    problema_hanoi = ProblemHanoi(estado_inicial, estado_objetivo)

    #Inicializamos el nodo raiz del arbol de busqueda 
    nodo_raiz = NodeHanoi(estado_inicial, None, None)

    #Empezamos con algoritmo de busqueda por anchura primero para tener un benchmark. (Sin optimizar) 
    #Ponemos nodo raiz en cola de nodos a explorar 
    varObjetivoCumplido = False
    work_queue_nodos = [nodo_raiz]

    while len(work_queue_nodos) != 0: 
        #Extraemos el primer nodo 
        nodo_actual = work_queue_nodos.pop(0)
        #Verificamos si nodo es objetivo. 
        if problema_hanoi.goal_test(nodo_actual.state): 
            print("Se ha encontrado el objetivo: ", nodo_actual.state)
            varObjetivoCumplido = True
            break
        #Si no es objetivo, tenemos que expandir el nodo
        else:
            lista_acciones_posibles = problema_hanoi.actions(nodo_actual.state)
            for accion in lista_acciones_posibles:
                #Creamos el nuevo estado a partir de la accion y el estado actual 
                nuevo_estado = problema_hanoi.result(nodo_actual.state, accion)
                #Creamos un nuevo nodo con el nuevo estado 
                nuevo_nodo = NodeHanoi(nuevo_estado, nodo_actual, accion)
                #Agregamos el nodo a la cola de nodos a explorar 
                work_queue_nodos.append(nuevo_nodo)

    if varObjetivoCumplido == False: 
        print("No se ha encontrado el objetivo en la busqueda por anchura")
        return None
    else: 
        print("El objetivo se ha encontrado en la busqueda por anchura")
        return nodo_actual



In [3]:
nodo = hanoi_busqueda_anchura_no_optimizada(disks=3)

Se ha encontrado el objetivo:  HanoiState:  |  | 3 2 1
El objetivo se ha encontrado en la busqueda por anchura


In [4]:

def hanoi_busqueda_anchura(disks=5):
    #Parametrizamos para ir aumentando discos 
    discos=disks

    #Seteamos estado inicial 
    varilla_1 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    varilla_2 = [] 
    varilla_3 = [] 
    estado_inicial = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Seteamos estado final 
    varilla_1 = []
    varilla_2 = [] 
    varilla_3 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    estado_objetivo = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Inicializamos la clase problema de hanoi con estado inicial y objetivo 
    problema_hanoi = ProblemHanoi(estado_inicial, estado_objetivo)

    #Inicializamos el nodo raiz del arbol de busqueda 
    nodo_raiz = NodeHanoi(estado_inicial, None, None)

    #Empezamos con algoritmo de busqueda por anchura primero para tener un benchmark. (Optimizado con set de nodos chequeados) 
    #Ponemos nodo raiz en cola de nodos a explorar 
    varObjetivoCumplido = False
    #work queue de nodos a explorar
    work_queue_nodos = [nodo_raiz]
    #nodos_chequeados son los estados ya verificados para no volver a explorarlos
    nodos_chequeados = set()

    while len(work_queue_nodos) != 0: 
        #Extraemos el primer nodo 
        nodo_actual = work_queue_nodos.pop(0)
        nodos_chequeados.add(nodo_actual.state)
        #Verificamos si nodo es objetivo. 
        if problema_hanoi.goal_test(nodo_actual.state): 
            varObjetivoCumplido = True
            metrics = {
            "solution_found": True,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
            return nodo_actual, metrics
            break
        #Si no es objetivo, tenemos que expandir el nodo
        else:
            #Extraemos posibles acciones
            lista_acciones_posibles = problema_hanoi.actions(nodo_actual.state)
            #Vemos resultado de cada accion y agregamos a la cola si no esta explorado
            for accion in lista_acciones_posibles:
                #Creamos el nuevo estado a partir de la accion y el estado actual 
                nuevo_estado = problema_hanoi.result(nodo_actual.state, accion)
                #Chequeamos si el estado ya fue chequeado. Si lo fue, seguimos al siguiente estado sin agregarlo a la cola de trabajo
                if nuevo_estado in nodos_chequeados: 
                    continue
                #Creamos un nuevo nodo con el nuevo estado 
                nuevo_nodo = NodeHanoi(nuevo_estado, nodo_actual, accion)
                #Agregamos el nodo a la cola de nodos a explorar 
                work_queue_nodos.append(nuevo_nodo)

    if varObjetivoCumplido == False: 
        print("No se ha encontrado el objetivo en la busqueda por anchura")
        metrics = {
            "solution_found": False,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
        return None, metrics

In [5]:
solution, metrics = hanoi_busqueda_anchura(disks=5)
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


In [6]:
#Camino completo 
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

In [7]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


In [8]:
import tracemalloc

tracemalloc.start()

solution, metrics = hanoi_busqueda_anchura(disks=7)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 27.27 [MB]


In [9]:
#Generar jsons para simulador
solution.generate_solution_for_simulator()

In [10]:
def hanoi_busqueda_profundidad(disks=5):
    #Parametrizamos para ir aumentando discos 
    discos=disks

    #Seteamos estado inicial 
    varilla_1 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    varilla_2 = [] 
    varilla_3 = [] 
    estado_inicial = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Seteamos estado final 
    varilla_1 = []
    varilla_2 = [] 
    varilla_3 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    estado_objetivo = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Inicializamos la clase problema de hanoi con estado inicial y objetivo 
    problema_hanoi = ProblemHanoi(estado_inicial, estado_objetivo)

    #Inicializamos el nodo raiz del arbol de busqueda 
    nodo_raiz = NodeHanoi(estado_inicial, None, None)

    #Ponemos nodo raiz en cola de nodos a explorar 
    varObjetivoCumplido = False
    #work queue de nodos a explorar
    work_queue_nodos = [nodo_raiz]
    #nodos_chequeados son los estados ya verificados para no volver a explorarlos
    nodos_chequeados = set()

    while len(work_queue_nodos) != 0: 
        #Extraemos el primer nodo 
        nodo_actual = work_queue_nodos.pop(0)
        nodos_chequeados.add(nodo_actual.state)
        #Verificamos si nodo es objetivo. 
        if problema_hanoi.goal_test(nodo_actual.state): 
            varObjetivoCumplido = True
            metrics = {
            "solution_found": True,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
            return nodo_actual, metrics
            break
        #Si no es objetivo, tenemos que expandir el nodo
        else:
            lista_acciones_posibles = problema_hanoi.actions(nodo_actual.state)
            for accion in lista_acciones_posibles:
                #Creamos el nuevo estado a partir de la accion y el estado actual 
                nuevo_estado = problema_hanoi.result(nodo_actual.state, accion)
                #Chequeamos si el estado ya fue chequeado. Si lo fue, seguimos al siguiente estado sin agregarlo a la cola de trabajo
                if nuevo_estado in nodos_chequeados: 
                    continue
                #Creamos un nuevo nodo con el nuevo estado 
                nuevo_nodo = NodeHanoi(nuevo_estado, nodo_actual, accion)
                #Aca cambiamos el append por un insert al principio de la lista para cambiar a búsqueda por profundidad
                work_queue_nodos.insert(0, nuevo_nodo)

    if varObjetivoCumplido == False: 
        print("No se ha encontrado el objetivo en la busqueda por anchura")
        metrics = {
            "solution_found": False,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
        return None, metrics

In [11]:
#Por profundidad
solution, metrics = hanoi_busqueda_profundidad(disks=5)

In [12]:
#Camino completo 
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 |  | 2 1>
<Node HanoiState: 5 4 3 1 |  | 2>
<Node HanoiState: 5 4 3 1 | 2 | >
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 1 |  | 3>
<Node HanoiState: 5 4 2 1 | 3 | >
<Node HanoiState: 5 4 2 | 3 | 1>
<Node HanoiState: 5 4 2 | 3 1 | >
<Node HanoiState: 5 4 | 3 1 | 2>
<Node HanoiState: 5 4 | 3 | 2 1>
<Node HanoiState: 5 4 1 | 3 | 2>
<Node HanoiState: 5 4 1 | 3 2 | >
<Node HanoiState: 5 4 | 3 2 | 1>
<Node HanoiState: 5 4 | 3 2 1 | >
<Node HanoiState: 5 | 3 2 1 | 4>
<Node HanoiState: 5 | 3 2 | 4 1>
<Node HanoiState: 5 1 | 3 2

In [13]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 4 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from

In [14]:
import tracemalloc

tracemalloc.start()

solution, metrics = hanoi_busqueda_anchura(disks=7)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 27.41 [MB]


In [15]:
#Generar jsons para simulador
solution.generate_solution_for_simulator()

In [16]:
def hanoi_busqueda_profundidad_limitada(disks=5, limite=100):
    #Parametrizamos para ir aumentando discos 
    discos=disks

    #Seteamos estado inicial 
    varilla_1 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    varilla_2 = [] 
    varilla_3 = [] 
    estado_inicial = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Seteamos estado final 
    varilla_1 = []
    varilla_2 = [] 
    varilla_3 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    estado_objetivo = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Inicializamos la clase problema de hanoi con estado inicial y objetivo 
    problema_hanoi = ProblemHanoi(estado_inicial, estado_objetivo)

    #Inicializamos el nodo raiz del arbol de busqueda 
    nodo_raiz = NodeHanoi(estado_inicial, None, None)

    #Ponemos nodo raiz en cola de nodos a explorar 
    varObjetivoCumplido = False
    #work queue de nodos a explorar
    work_queue_nodos = [nodo_raiz]
    #nodos_chequeados son los estados ya verificados para no volver a explorarlos
    nodos_chequeados = set()

    while len(work_queue_nodos) != 0: 
        #Extraemos el primer nodo 
        nodo_actual = work_queue_nodos.pop(0)
        nodos_chequeados.add(nodo_actual.state)
        #Verificamos si nodo es objetivo. 
        if problema_hanoi.goal_test(nodo_actual.state): 
            varObjetivoCumplido = True
            metrics = {
            "solution_found": True,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
            return nodo_actual, metrics
        #Si no es objetivo, tenemos que expandir el nodo
        else:
            lista_acciones_posibles = problema_hanoi.actions(nodo_actual.state)
            for accion in lista_acciones_posibles:
                #Creamos el nuevo estado a partir de la accion y el estado actual 
                nuevo_estado = problema_hanoi.result(nodo_actual.state, accion)
                #Creamos un nuevo nodo con el nuevo estado 
                nuevo_nodo = NodeHanoi(nuevo_estado, nodo_actual, accion)
                #Chequeamos si el estado ya fue chequeado. Si lo fue, seguimos al siguiente estado sin agregarlo a la cola de trabajo
                if (nuevo_estado in nodos_chequeados) or nuevo_nodo.depth>limite: 
                    continue
                else: 
                    #Aca cambiamos el append por un insert al principio de la lista para cambiar a búsqueda por profundidad
                    work_queue_nodos.insert(0, nuevo_nodo)

    if varObjetivoCumplido == False: 
        print(f"No se ha encontrado el objetivo en la búsqueda limitada a {limite} niveles")
        metrics = {
            "solution_found": False,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "max_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            }
        return None, metrics

In [17]:
solution, metrics = hanoi_busqueda_profundidad_limitada(disks=5, limite=120)

No se ha encontrado el objetivo en la búsqueda limitada a 120 niveles


In [18]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: False
states_visited: 241
nodes_in_frontier: 0
max_depth: 1
cost_total: 1.0


In [19]:
def hanoi_busqueda_profundidad_limitada_v2(disks=5, limite=100):
    #Parametrizamos para ir aumentando discos 
    discos=disks

    #Seteamos estado inicial 
    varilla_1 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    varilla_2 = [] 
    varilla_3 = [] 
    estado_inicial = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Seteamos estado final 
    varilla_1 = []
    varilla_2 = [] 
    varilla_3 = list(range(discos, 0, -1))  # Discos en la varilla 1, de mayor a menor
    estado_objetivo = StatesHanoi(varilla_1, varilla_2, varilla_3, max_disks=discos) 

    #Inicializamos la clase problema de hanoi con estado inicial y objetivo 
    problema_hanoi = ProblemHanoi(estado_inicial, estado_objetivo)

    #Inicializamos el nodo raiz del arbol de busqueda 
    nodo_raiz = NodeHanoi(estado_inicial, None, None)

    #Ponemos nodo raiz en cola de nodos a explorar 
    varObjetivoCumplido = False
    #work queue de nodos a explorar
    work_queue_nodos = [nodo_raiz]
    #nodos_chequeados son los estados ya verificados para no volver a explorarlos
    nodos_chequeados = dict()
    max_depth_reached = 0

    while len(work_queue_nodos) != 0: 
        #Extraemos el primer nodo 
        nodo_actual = work_queue_nodos.pop(0)

        # Si ya lo visitamos a una profundidad menor o igual, lo ignoramos
        if nodo_actual.state in nodos_chequeados and nodo_actual.depth >= nodos_chequeados[nodo_actual.state]:
            continue
        # Guardamos la mejor profundidad alcanzada
        nodos_chequeados[nodo_actual.state] = nodo_actual.depth

        #Verificamos si nodo es objetivo.
        max_depth_reached = max(max_depth_reached, nodo_actual.depth)
        if problema_hanoi.goal_test(nodo_actual.state): 
            varObjetivoCumplido = True
            metrics = {
            "solution_found": True,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "solution_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            "max_depth": max_depth_reached,
            }
            return nodo_actual, metrics, nodos_chequeados
        #Si no es objetivo, tenemos que expandir el nodo
        else:
            lista_acciones_posibles = problema_hanoi.actions(nodo_actual.state)
            for accion in lista_acciones_posibles:
                #Creamos el nuevo estado a partir de la accion y el estado actual 
                nuevo_estado = problema_hanoi.result(nodo_actual.state, accion)
                #Creamos un nuevo nodo con el nuevo estado 
                nuevo_nodo = NodeHanoi(nuevo_estado, nodo_actual, accion)
                #Chequeamos si el estado ya fue chequeado. Si lo fue, seguimos al siguiente estado sin agregarlo a la cola de trabajo
                if nuevo_nodo.depth > limite:
                    continue
                if nuevo_nodo.state in nodos_chequeados and nuevo_nodo.depth >= nodos_chequeados[nuevo_nodo.state]:
                    continue
                work_queue_nodos.insert(0, nuevo_nodo)                                      

    if varObjetivoCumplido == False: 
        print(f"No se ha encontrado el objetivo en la búsqueda limitada a {limite} niveles")
        metrics = {
            "solution_found": False,
            "states_visited": len(nodos_chequeados),
            "nodes_in_frontier": len(work_queue_nodos),
            "solution_depth": nodo_actual.depth,
            "cost_total": nodo_actual.state.accumulated_cost,
            "max_depth": max_depth_reached,
            }
        return None, metrics, nodos_chequeados

In [20]:
solution, metrics, lista_de_nodos = hanoi_busqueda_profundidad_limitada_v2(disks=8, limite=1000000)

In [21]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
states_visited: 3281
nodes_in_frontier: 1644
solution_depth: 3280
cost_total: 3280.0
max_depth: 3280


In [22]:
solution, metrics = hanoi_busqueda_anchura(disks=5)

In [23]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


In [24]:
lista_de_nodos

{HanoiState: 8 7 6 5 4 3 2 1 |  | : 0,
 HanoiState: 8 7 6 5 4 3 2 |  | 1: 1,
 HanoiState: 8 7 6 5 4 3 2 | 1 | : 2,
 HanoiState: 8 7 6 5 4 3 | 1 | 2: 3,
 HanoiState: 8 7 6 5 4 3 |  | 2 1: 4,
 HanoiState: 8 7 6 5 4 3 1 |  | 2: 5,
 HanoiState: 8 7 6 5 4 3 1 | 2 | : 6,
 HanoiState: 8 7 6 5 4 3 | 2 | 1: 7,
 HanoiState: 8 7 6 5 4 3 | 2 1 | : 8,
 HanoiState: 8 7 6 5 4 | 2 1 | 3: 9,
 HanoiState: 8 7 6 5 4 | 2 | 3 1: 10,
 HanoiState: 8 7 6 5 4 1 | 2 | 3: 11,
 HanoiState: 8 7 6 5 4 1 |  | 3 2: 12,
 HanoiState: 8 7 6 5 4 |  | 3 2 1: 13,
 HanoiState: 8 7 6 5 4 | 1 | 3 2: 14,
 HanoiState: 8 7 6 5 4 2 | 1 | 3: 15,
 HanoiState: 8 7 6 5 4 2 |  | 3 1: 16,
 HanoiState: 8 7 6 5 4 2 1 |  | 3: 17,
 HanoiState: 8 7 6 5 4 2 1 | 3 | : 18,
 HanoiState: 8 7 6 5 4 2 | 3 | 1: 19,
 HanoiState: 8 7 6 5 4 2 | 3 1 | : 20,
 HanoiState: 8 7 6 5 4 | 3 1 | 2: 21,
 HanoiState: 8 7 6 5 4 | 3 | 2 1: 22,
 HanoiState: 8 7 6 5 4 1 | 3 | 2: 23,
 HanoiState: 8 7 6 5 4 1 | 3 2 | : 24,
 HanoiState: 8 7 6 5 4 | 3 2 | 1: 25,
 HanoiS